<a href="https://colab.research.google.com/github/amanjain1397/zip-the-code-1.0/blob/master/v_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re
import keras

import nltk 
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

/home/ipmob/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ipmob/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def preprocess_text(text):
    
    #Lowering case the text
    text = text.lower()
    
    #Removing numbers
    text = re.sub(r'\d+', '', text)
    
    #Removing punctuations
    import string
    from nltk.tokenize import word_tokenize
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Removing stop words
    
    tokens = word_tokenize(text)
    text = ' '.join([ i for i in tokens if not i in stopwords])
    
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return text

In [3]:
data = pd.read_csv('data/train_file.csv')
test = pd.read_csv('data/test_file.csv')
sample = pd.read_csv('data/results_file.csv')

data.loc[data.Subjects.isnull(), 'Subjects'] = ''
test.loc[test.Subjects.isnull(), 'Subjects'] = ''

data['combined'] = data.Subjects.str.cat(' ' + data.Title)
test['combined'] = test.Subjects.str.cat(' ' + test.Title)

dataMatType = data.MaterialType
data.drop(labels = ['MaterialType'], axis = 1, inplace = True)

data.combined = data.combined.apply(preprocess_text)
test.combined = test.combined.apply(preprocess_text)
'''
data.Subjects = data.Subjects.apply(preprocess_text)
data.Title = data.Title.apply(preprocess_text)
test.Subjects = test.Subjects.apply(preprocess_text)
test.Title = test.Title.apply(preprocess_text)
'''
flag = -1
labelDict = {}

for key in dataMatType.value_counts().keys():
  flag += 1
  labelDict[key] = flag
  
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 10                             

max_words = 1000                                    

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.combined)

dataSequences = tokenizer.texts_to_sequences(data.combined)
testSequences = tokenizer.texts_to_sequences(test.combined)

data_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(data_word_index))

Found 36088 unique tokens.


In [4]:
from keras import preprocessing
x_train = preprocessing.sequence.pad_sequences(dataSequences, maxlen = 10)
x_test = preprocessing.sequence.pad_sequences(testSequences, maxlen = 10)

train_label = dataMatType.map(labelDict)

from keras.utils import to_categorical
labelDataBinary = to_categorical(train_label)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(dataMatType), dataMatType)

In [5]:
import os


embeddings_index = {}
f = open('../glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in data_word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [6]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 300, input_length=10))

'''model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
'''


model.add(Conv1D(128, 3,  activation='relu'))
model.add(Dropout(0.3))

model.add(LSTM(400, recurrent_dropout = 0.6, return_sequences = True))
model.add(Dropout(0.6))

model.add(LSTM(400, recurrent_dropout = 0.2))
model.add(Dropout(0.2))

model.add(Dense(8, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, labelDataBinary,
                    epochs=8,
                    validation_split=0.2,
                    class_weight = class_weights)

model.save('m1.h5')

Train on 25322 samples, validate on 6331 samples
Epoch 1/8
25322/25322 [==============================] - 69s 3ms/step - loss: 0.5867 - acc: 0.8380 - val_loss: 0.4669 - val_acc: 0.8728
Epoch 2/8
25322/25322 [==============================] - 52s 2ms/step - loss: 0.4618 - acc: 0.8700 - val_loss: 0.4384 - val_acc: 0.8755
Epoch 3/8
25322/25322 [==============================] - 53s 2ms/step - loss: 0.4402 - acc: 0.8750 - val_loss: 0.4419 - val_acc: 0.8825
Epoch 4/8
25322/25322 [==============================] - 54s 2ms/step - loss: 0.4262 - acc: 0.8784 - val_loss: 0.4355 - val_acc: 0.8825
Epoch 5/8
25322/25322 [==============================] - 53s 2ms/step - loss: 0.4172 - acc: 0.8786 - val_loss: 0.4262 - val_acc: 0.8814
Epoch 6/8
25322/25322 [==============================] - 54s 2ms/step - loss: 0.4124 - acc: 0.8816 - val_loss: 0.4395 - val_acc: 0.8820
Epoch 7/8
25322/25322 [==============================] - 55s 2ms/step - loss: 0.4045 - acc: 0.8826 - val_loss: 0.4416 - val_acc: 0.8823

In [8]:
#model.save('./models/m1.h5')
model.save('m1.h5')

**Using Embedding Layer**

In [7]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 300, input_length=10, weights=[embedding_matrix], trainable=False))

'''model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
'''

model.add(LSTM(1000, recurrent_dropout = 0.3))
model.add(Dropout(0.35))
model.add(Dense(8, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, labelDataBinary,
                    epochs=8,
                    batch_size=128,
                    validation_split=0.2,
                    class_weight = class_weights)

Train on 25322 samples, validate on 6331 samples
Epoch 1/8
25322/25322 [==============================] - 45s 2ms/step - loss: 0.5646 - acc: 0.8440 - val_loss: 0.4672 - val_acc: 0.8574
Epoch 2/8
25322/25322 [==============================] - 42s 2ms/step - loss: 0.4551 - acc: 0.8647 - val_loss: 0.4274 - val_acc: 0.8686
Epoch 3/8
25322/25322 [==============================] - 41s 2ms/step - loss: 0.4218 - acc: 0.8710 - val_loss: 0.4192 - val_acc: 0.8711
Epoch 4/8
25322/25322 [==============================] - 40s 2ms/step - loss: 0.3957 - acc: 0.8754 - val_loss: 0.4168 - val_acc: 0.8724
Epoch 5/8
25322/25322 [==============================] - 45s 2ms/step - loss: 0.3713 - acc: 0.8809 - val_loss: 0.4109 - val_acc: 0.8768
Epoch 6/8
25322/25322 [==============================] - 45s 2ms/step - loss: 0.3469 - acc: 0.8870 - val_loss: 0.4196 - val_acc: 0.8751
Epoch 7/8
25322/25322 [==============================] - 49s 2ms/step - loss: 0.3189 - acc: 0.8929 - val_loss: 0.4198 - val_acc: 0.8757

In [9]:
reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
  
predictions = model.predict(x_test)
predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
predictions = predictions.map(reverseLabelDict)

submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']

In [12]:
submission.to_csv('sub-dl.csv', index = False, header = True)

In [ ]:
def predict(model, x_test):
  predictions = model.predict(x_test)
  predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
  predictions = predictions.map(reverseLabelDict)
  return predictions

In [ ]:
#num_model = 5
predic1 = pd.Series(predict(model1, x_test))
#predic2 = pd.Series(predict(model2, x_test))
#predic3 = pd.Series(predict(model3, x_test))
#predic4 = pd.Series(predict(model4, x_test))
#predic5 = pd.Series(predict(model5, x_test))

In [ ]:
#total = pd.DataFrame(pd.concat([predic1, predic2, predic3, predic4, predic5], axis = 1))
total = pd.DataFrame(pd.concat([predic1], axis = 1))
submissions = total.mode(axis = 1)[0]
submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('sub-mew.csv', index = False, header = True)

In [ ]:
submissions = total.mode(axis = 1)[0]